In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import string
import pickle
import emoji
import scipy.stats as ss
import sklearn
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report
import nltk
nltk.download('omw-1.4')

%matplotlib inline

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Kamen\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
lemmatizer = WordNetLemmatizer()
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

#Convert emojis to words 
with open('Emoji_Dict.p', 'rb') as fp:
        Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def convert_emojis_to_word(mess):
    for emot in Emoji_Dict:
        mess = re.sub(r'('+emot+')', "_".join(Emoji_Dict[emot].replace(",","").replace(":","").split()), mess)
    return mess

def text_process(mess):
    
    #Convert emojis to words
    mess = convert_emojis_to_word(mess)
    mess = mess.lower() # lowercase text
    mess = REPLACE_BY_SPACE_RE.sub('', mess) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    mess = BAD_SYMBOLS_RE.sub('', mess)

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    stop = ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

    return stop

In [3]:
text_process('Hello, my name is Kamen. I hope you have a good day.')

'hello name kamen hope good day'

In [4]:
#A NEW DAY, NEW OPPORTUNITY
df1 = pd.read_csv('train.txt',sep=';')
df1.rename({'i didnt feel humiliated':'text','sadness':'label'}, axis = 1, inplace=True)
df2 = pd.read_csv('val.txt', sep=';')
df2.rename({'im feeling quite sad and sorry for myself but ill snap out of it soon':'text','sadness':'label'}, axis = 1, inplace=True)
df = pd.concat([df1,df2], axis=0) 


In [5]:
df.rename({'i didnt feel humiliated':'text','sadness':'label'}, axis = 1, inplace=True)

In [6]:
len(df)

17998

In [7]:
#bow_transformer = CountVectorizer(analyzer=text_process).fit(df['text'])
#text_bow = bow_transformer.transform(df['text'])

In [8]:
#Train/test splits
msg_train, msg_test, label_train, label_test = \
train_test_split(df['text'], df['label'], test_size=0.2, stratify=df['label'])

print(len(msg_train), len(msg_test), len(msg_train) + len(msg_test))

vect = CountVectorizer()
msg_train_vect = vect.fit_transform(msg_train)
msg_test_vect = vect.fit_transform(msg_test)

14398 3600 17998


In [9]:
#LSTM
from keras.utils.data_utils import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding, Bidirectional

In [10]:
df['to_check'] = df['text'].apply(text_process)